In [1]:
%pip install transformers
%pip install torch
%pip install nltk
%pip install contractions

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import contractions
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, BertConfig

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

/home/codespace/.python/current/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-


True

### Dataset

In [4]:
df = pd.read_csv('data/Aug_9_labeling_500_df_sampled.csv')
df.head()

,origin,Unnamed: 0,id,text,CreatedAt,author.userName,createdDate,misinformation,stance,isUSRelated,notes
0,T2,1224387,1.479670e+18,Fox News: Harris comms chief apologizes after ...,Sat Jan 08 04:12:37 +0000 2022,01splcheck,1/7/22,true information,neutral,NaN,NaN
1,T4,332659,1.778180e+18,@Cartel_Cal @RepRobertGarcia It seems he's in ...,Wed Apr 10 22:02:07 +0000 2024,01splcheck,4/10/24,opinion,anti,NaN,NaN
2,T3,1567887,1.656440e+18,@NatlyDenise_ Ready to bring illegal immigrant...,Wed May 10 23:27:26 +0000 2023,01splcheck,5/10/23,opinion,anti,NaN,NaN
3,T2,1303080,1.477880e+18,"@cassstastrophe Well, you can take comfort tha...",Mon Jan 03 06:00:21 +0000 2022,01splcheck,1/2/22,misleading content,anti,NaN,NaN
4,T3,1578404,1.656440e+18,I see a press conference with Arizona official...,Wed May 10 23:31:22 +0000 2023,01splcheck,5/10/23,uncertain,anti,NaN,"true information, or misleading content, no ve..."


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2829 entries, 0 to 2828
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   origin           2829 non-null   object 
 1   Unnamed: 0       2829 non-null   int64  
 2   id               2829 non-null   float64
 3   text             2829 non-null   object 
 4   CreatedAt        2829 non-null   object 
 5   author.userName  2829 non-null   object 
 6   createdDate      2829 non-null   object 
 7   misinformation   487 non-null    object 
 8   stance           487 non-null    object 
 9   isUSRelated      180 non-null    object 
 10  notes            104 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 243.2+ KB


In [6]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Expand contractions 
    text = contractions.fix(text)
    
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # Remove mentions 
    text = re.sub(r'@\w+', '', text)
    
    # Remove emojis (using ASCII encoding)
    text = text.encode('ascii', 'ignore').decode('ascii')
    
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    
    # Tokenize the text
    tokens = nltk.word_tokenize(text)
    
    # Part-of-Speech tagging
    pos_tags = nltk.pos_tag(tokens)
    
    # Process tokens: lowercase if not NNP or NNPS, remove stopwords, and lemmatize
    processed_tokens = []
    for token, tag in pos_tags:
        if tag not in ('NNP', 'NNPS'):
            token = token.lower()
        if token not in stop_words:
            token = lemmatizer.lemmatize(token)
            processed_tokens.append(token)
    
    # Join tokens back to string
    cleaned_text = ' '.join(processed_tokens)
    
    return cleaned_text

In [10]:
df['cleaned_text'] = df['text'].apply(preprocess_text)
df.to_csv('data/processed_t2-4.csv')

### Model

In [11]:
# load the model weights
state_dict = torch.load("fakeBert.bin", map_location=torch.device('cpu'))
config = BertConfig.from_pretrained("bert-base-uncased", num_labels=2)
model = AutoModelForSequenceClassification.from_pretrained(
    pretrained_model_name_or_path=None, config=config, state_dict=state_dict
)

# load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

/tmp/ipykernel_2208/2621768022.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load("fakeBert.bin", map_location=torch.device('cpu'))
Some weights of 

In [12]:
# predictions
def predict(text):
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=-1).item()
    return "True" if prediction == 1 else "False"

df['prediction'] = df['cleaned_text'].apply(predict)
df['prediction'].value_counts()

prediction
True     2798
False      31
Name: count, dtype: int64